In [11]:
import numpy as np
import revreaddy as rdy
import logging
import datetime
import h5py as h5
import matplotlib.cm as cm
import time
import random
import string

import matplotlib.pyplot as plt
reload(logging)
logging.basicConfig(
     format='%(asctime)s %(levelname)s: %(message)s',
     datefmt='%Y/%m/%d %I:%M:%S',
     level=logging.DEBUG
    )

In [12]:
def stamp():
    
    
    timestamp = time.strftime("%Y_%m_%d-%H_%M_%S")
    randomstamp = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(6))
    stamp = timestamp + "_" + randomstamp
    return stamp


In [13]:
def simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim()
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.timestep = 0.01 # ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0., reactiondistance)
    sim.delete_all_particles()
    np.random.seed()

    for _ in range(200):
         pos = np.random.random(3) * 9.999 - 4.999
         sim.add_particle(pos, 0)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 1)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 2)
    for _ in range(1):   
        pos = np.random.random(3) * 9.9 - 4.95
        sim.add_particle(pos, 2)
    sim.show_world()
    sim.delete_all_observables()
    d = datetime.datetime.now()
    time=str(getattr(d, 'year','month'))+str(getattr(d,'month'))+str(getattr(d,'day'))+str(getattr(d,'hour'))+str(getattr(d,'minute'))+str(getattr(d,'second'))
    numbers_a_name = time+"numbers_a_test1.h5"
    sim.new_particle_numbers(1, numbers_a_name, 0)
    numbers_b_name = time+"numbers_b_test1.h5"
    mean_e_name = time+"mean_e_test1.h5"
    sim.new_particle_numbers(1, numbers_b_name, 1)
    sim.new_mean_squared_displacement(1,mean_e_name, 2)
    sim.run(1000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fa = h5.File(numbers_a_name, 'r')
    fb = h5.File(numbers_b_name, 'r')
    fc = h5.File(mean_e_name, 'r')
    return fa , fb ,fc

In [14]:
def simulationfrac(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim("FractionalDiffusion")
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.timestep = 0.01 # ns
    sim.alpha = 0.5
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0., reactiondistance)
    sim.delete_all_particles()
    np.random.seed()

    for _ in range(200):
         pos = np.random.random(3) * 9.999 - 4.999
         sim.add_particle(pos, 0)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 1)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 2)
    for _ in range(1):   
        pos = np.random.random(3) * 9.9 - 4.95
        sim.add_particle(pos, 2)
    sim.show_world()
    sim.delete_all_observables()
   
    numbers_a_name = stamp()+"numbers_a_test1.h5"
    sim.new_particle_numbers(1, numbers_a_name, 0)
    sim.
    numbers_b_name = stamp()+"numbers_b_test1.h5"
    mean_e_name = stamp()+"mean_e_test1.h5"
    sim.new_particle_numbers(1, numbers_b_name, 1)
    sim.new_mean_squared_displacement(1,mean_e_name, 2)
    sim.run(1000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fa = h5.File(numbers_a_name, 'r')
    fb = h5.File(numbers_b_name, 'r')
    fc = h5.File(mean_e_name, 'r')
    return fa , fb ,fc

In [15]:
#hier kommt die berechnung des verhältnisses der reactionskoeffizienten und der Concentration vom enzym
def analyt(t,boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction):
    #a=((4*np.pi*reactiondistance**3)*(Diffusion_E+Diffusion_S))/((3*boxsize**3)*(Diffusion_E+Diffusion_P))
    k1=4*np.pi*2*(Diffusion_E+Diffusion_S)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_S)/intrinsikreaction)+np.tanh(reactiondistance*np.sqrt(intrinsikreaction/(Diffusion_E+Diffusion_S))))
    #t=np.linspace(1,40)
    #vor und rück reaction möglich                                        
    #c=np.exp(-t*a)*fa['particleNumbers'][0]
    #nur vor reaction
    k11=intrinsikreaction*4*np.pi*reactiondistance**3/3
    c=fa['particleNumbers'][2]*(np.exp((-t*k1*1)/(boxsize**3)))
    c11=fa['particleNumbers'][0]*np.exp((-t*k11*1)/(boxsize**3))

    return t, c,c11

In [16]:
def msd_analyt(alpha,dt,t,diffconst):
    t=np.linspace(0,t,t)
    return 2*3*((t*dt)**alpha)*diffconst

In [21]:
#colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
for i in [1.5]:
    boxsize=10.
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=10
    reactiondistance=3
    intrinsikreaction=10000000000000000000000
    #colornow=next(colors)
    #falist=[]
    #fafraclist=[]
    fclist=[]
    fcfraclist=[]

    averagelist=[]
    for ii in range(50):
        fa,fb,fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction)
        fafrc,fbfrc,fcfrc=simulationfrac(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction)
        #plt.plot(fa['times'], fa['particleNumbers'],color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
        #falist.append(fa['particleNumbers'])
        #fafraclist.append(fafrc['particleNumbers'])
        #t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
        #averagelist.append(c)
        fclist.append(fc['meanSquaredDisplacements'])
        fcfraclist.append(fcfrc['meanSquaredDisplacements'])


    #averagelist=np.array(averagelist) 
    #analytnumer=averagelist.mean(axis=0)
    #falist=np.array(falist)  
    #fafraclist=np.array(fafraclist)  
    #t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
    #paertnum=falist.mean(axis=0) 
    #paertnumfrac=fafraclist.mean(axis=0)  
    fclist=np.array(fclist)
    fcfraclist=np.array(fcfraclist)  


    msd=fclist.mean(axis=0) 
    msdfrc=fcfraclist.mean(axis=0) 
    plt.plot(fc['times'], msd ,label="$D_E$=$%.2f$" %(i),color='b')
    plt.plot(np.array(range(1000))*0.01,msd_analyt(1.0,0.01,1000,1.5),label="ana",color='b')

    plt.plot(fc['times'], msdfrc ,label="$D_E$=$%.2f$" %(i),marker=',',color='r')
    plt.plot(np.array(range(1000))*0.01,msd_analyt(0.5,0.01,1000,1.5),label="anafrc",color='r')


    
    #plt.plot(fafrc['times'], paertnumfrac,color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i),marker=",")
    #plt.plot(fa['times'], paertnum,color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
    #plt.plot(t,c,"--",color=colornow)
plt.ylabel('MSD)', fontsize=10)
plt.xlabel('t in ns', fontsize=10)
plt.legend()

2016/07/14 05:00:35 INFO: There are either no particles or too much (>200) to list them here.
2016/07/14 05:00:35 INFO: Run with timestep 0.01 and 1000 timesteps
2016/07/14 05:00:41 INFO: Finished after 5.724001 seconds.


Number of particles:  201
TypeId | Number of particles
====== | ===================
0      | 200                
1      | 0                  
2      | 1                  
 
Number of particles: 

2016/07/14 05:00:41 INFO: There are either no particles or too much (>200) to list them here.
2016/07/14 05:00:41 INFO: Run with timestep 0.01 and 1000 timesteps
2016/07/14 05:00:47 INFO: Finished after 6.6641 seconds.


 201
TypeId | Number of particles
====== | ===================
0      | 200                
1      | 0                  
2      | 1                  
 
Number of particles: 

2016/07/14 05:00:47 INFO: There are either no particles or too much (>200) to list them here.
2016/07/14 05:00:47 INFO: Run with timestep 0.01 and 1000 timesteps
2016/07/14 05:00:53 INFO: Finished after 5.749222 seconds.


...

 201
TypeId | Number of particles
====== | ===================
0      | 200                
1      | 0                  
2      | 1                  
 
Number of particles: 

2016/07/14 05:10:49 INFO: There are either no particles or too much (>200) to list them here.
2016/07/14 05:10:49 INFO: Run with timestep 0.01 and 1000 timesteps
2016/07/14 05:10:54 INFO: Finished after 5.80327 seconds.


 201
TypeId | Number of particles
====== | ===================
0      | 200                
1      | 0                  
2      | 1                  
 
Number of particles: 

2016/07/14 05:10:55 INFO: There are either no particles or too much (>200) to list them here.
2016/07/14 05:10:55 INFO: Run with timestep 0.01 and 1000 timesteps
2016/07/14 05:11:01 INFO: Finished after 6.729391 seconds.


 201
TypeId | Number of particles
====== | ===================
0      | 200                
1      | 0                  
2      | 1                  
 


In [22]:
plt.show()

In [ ]:
colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
for i in [1.5,2.,3.5,5.]:
    boxsize=10.
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=10
    reactiondistance=3
    intrinsikreaction=10000000000000000000000
    colornow=next(colors)
    falist=[]
    fafraclist=[]
    #fclist=[]
    #fcfraclist=[]

    averagelist=[]
    for ii in range(5):
        fa,fb,fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction)
        fafrc,fbfrc,fcfrc=simulationfrac(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction)
        #plt.plot(fa['times'], fa['particleNumbers'],color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
        falist.append(fa['particleNumbers'])
        fafraclist.append(fafrc['particleNumbers'])
        t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
        averagelist.append(c)
        #fclist.append(fc['meanSquaredDisplacements'])
        #fcfraclist.append(fcfrc['meanSquaredDisplacements'])


    averagelist=np.array(averagelist) 
    analytnumer=averagelist.mean(axis=0)
    falist=np.array(falist)  
    fafraclist=np.array(fafraclist)  

    t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
    paertnum=falist.mean(axis=0) 
    paertnumfrac=fafraclist.mean(axis=0)  
    #msd=fclist.mean(axis=0) 

    #msdfrc=fcfraclist.mean(axis=0) 

    
    plt.plot(fafrc['times'], paertnumfrac,color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i),marker=",")
    plt.plot(fa['times'], paertnum,color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
    plt.plot(t,c,"--",color=colornow)
plt.ylabel('S(t)', fontsize=10)
plt.xlabel('t in ns', fontsize=10)
plt.legend()